In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path

In [3]:
!ls /kaggle/input/processed-yelp-dataset/data/

business_data.csv  review_testing.csv  review_training.csv  users_data.csv


In [4]:
# Fonction de nettoyage des données
def clean_data(df):
    # Supprimer les doublons selon user, business et note
    df = df.drop_duplicates(subset=["user_id", "business_id", "stars"])
    
    # Supprimer les lignes avec valeurs manquantes importantes
    df = df.dropna(subset=["user_id", "business_id", "stars", "text"])
    
    # Forcer le type int pour les notes
    df["stars"] = df["stars"].astype(int)
    
    return df

# Fonction de traitement par batch (chunks) CSV → Parquet
def process_csv_in_chunks(csv_path, parquet_path, chunksize=100000):
    writer = None  # Écrivain Parquet
    for chunk in pd.read_csv(
        csv_path,
        chunksize=chunksize,
        usecols=["review_id", "user_id", "business_id", "stars", "text" , "date"]
    ):
        # Nettoyage du batch
        clean_chunk = clean_data(chunk)

        # Conversion en table Apache Arrow
        table = pa.Table.from_pandas(clean_chunk)

        # Écriture dans le fichier Parquet
        if writer is None:
            writer = pq.ParquetWriter(parquet_path, table.schema, compression='snappy')
        writer.write_table(table)
    
    # Fermer l'écrivain proprement
    if writer:
        writer.close()
    
    print(f"Fichier sauvegardé : {parquet_path}")

# Traiter les deux fichiers CSV (training et testing)
process_csv_in_chunks(
    "/kaggle/input/processed-yelp-dataset/data/review_training.csv",
    "yelp_reviews_train_preprocessed.parquet"
)

process_csv_in_chunks(
    "/kaggle/input/processed-yelp-dataset/data/review_testing.csv",
    "yelp_reviews_test_preprocessed.parquet"
)

Fichier sauvegardé : yelp_reviews_train_preprocessed.parquet
Fichier sauvegardé : yelp_reviews_test_preprocessed.parquet


In [5]:
import pandas as pd

# Charger le fichier Parquet prétraité
train_reviews = pd.read_parquet("yelp_reviews_train_preprocessed.parquet")

# Nettoyage et filtrage
def clean_and_filter(df):
    # Supprimer les doublons sur [user_id, business_id, stars]
    df = df.drop_duplicates(subset=["user_id", "business_id", "stars"])
    
    # Supprimer les lignes avec valeurs manquantes importantes
    df = df.dropna(subset=["user_id", "business_id", "stars", "text"])
    
    # Supprimer seulement les reviews trop courtes (< 10 caractères)
    df = df[df['text'].str.len() >= 10]
    
    return df

train_reviews = clean_and_filter(train_reviews)

# Vérification
print("=== Vérification du train ===")
print("Shape :", train_reviews.shape)
print("Nombre de doublons sur [user_id, business_id, stars] :", 
      train_reviews.duplicated(subset=["user_id", "business_id", "stars"]).sum())
print("\nValeurs manquantes par colonne :")
print(train_reviews.isna().sum())
print("\nLongueur des textes :")
print(train_reviews['text'].str.len().describe())
print("\nAperçu :")
print(train_reviews.head())

=== Vérification du train ===
Shape : (4444812, 6)
Nombre de doublons sur [user_id, business_id, stars] : 0

Valeurs manquantes par colonne :
review_id      0
user_id        0
business_id    0
stars          0
text           0
date           1
dtype: int64

Longueur des textes :
count    4.444812e+06
mean     5.498948e+02
std      5.125874e+02
min      1.000000e+01
25%      2.230000e+02
50%      3.910000e+02
75%      6.930000e+02
max      5.000000e+03
Name: text, dtype: float64

Aperçu :
                review_id                 user_id             business_id  \
0  bwaSF4fYDBgyls0MBVcuOw  ZvmZM7LBzaYbpznpHlTANA  lmgCjE_h9VkpJYjsD2Li0A   
1  hOl6MoudNRyeK7iC8V_UvA  UMS5obm9Gwr7AyfvEJyhkw  fGYnHzFr1z2kv7bPRW6VMA   
2  VnVU0FtwvQtOHcB9unmVtg  9a0US1vaEBiQlMl2j2vmrg  7EhTT4iEuA7JaaWA-eI3Qw   
3  9uJavmwzNEy10xmvd4935A  bKG5aNvEBU8Ckzf8JbXjlA  fGYnHzFr1z2kv7bPRW6VMA   
4  TZ-WMedZ6YnYH1RpJF8CnA  bSAEDuyuE4-c3etP8_5lLA  TTsBa6e2qgIEkiFQfYLmYA   

   stars                                    

In [6]:
import pandas as pd

# Charger le fichier Parquet prétraité pour le test
test_reviews = pd.read_parquet("yelp_reviews_test_preprocessed.parquet")

# Nettoyage et filtrage (même fonction que pour le train)
def clean_and_filter(df):
    # Supprimer les doublons sur [user_id, business_id, stars]
    df = df.drop_duplicates(subset=["user_id", "business_id", "stars"])
    
    # Supprimer les lignes avec valeurs manquantes importantes
    df = df.dropna(subset=["user_id", "business_id", "stars", "text"])
    
    # Supprimer seulement les reviews trop courtes (< 15 caractères)
    df = df[df['text'].str.len() >= 13]
    
    return df

test_reviews = clean_and_filter(test_reviews)

# Vérification
print("=== Vérification du test ===")
print("Shape :", test_reviews.shape)
print("Nombre de doublons sur [user_id, business_id, stars] :", 
      test_reviews.duplicated(subset=["user_id", "business_id", "stars"]).sum())
print("\nValeurs manquantes par colonne :")
print(test_reviews.isna().sum())
print("\nLongueur des textes :")
print(test_reviews['text'].str.len().describe())
print("\nAperçu :")
print(test_reviews.head())

=== Vérification du test ===
Shape : (1155276, 6)
Nombre de doublons sur [user_id, business_id, stars] : 0

Valeurs manquantes par colonne :
review_id      0
user_id        0
business_id    0
stars          0
text           0
date           0
dtype: int64

Longueur des textes :
count    1.155276e+06
mean     5.087396e+02
std      4.788763e+02
min      1.300000e+01
25%      2.070000e+02
50%      3.620000e+02
75%      6.370000e+02
max      5.000000e+03
Name: text, dtype: float64

Aperçu :
                review_id                 user_id             business_id  \
0  5-tLhwWjSzHYN6NyMy9Suw  IlWLPCRQp8iqX0X-ExQccQ  bdfZdB2MTXlT6-RBjSIpQg   
1  4M72cXIgMLwXo8METuXZeQ  nfLL3vn_4-zXAfoh7hWiZg  MbzgGsMQpGyVrUJXi_Jw0Q   
2  oC0ADnj65JKMk5zu6FDz5w  kV4eLnU5fm-ieJCEeqNiIQ  ZY2feSm2sc5nTXYHnKpMGQ   
3  VpibERhdmOvSagzKSdVuSw  sWVVqJQAl6w6cOzbUsUC3A  tsx84z4c0B-y6J5fqfvBqg   
4  1HmF9bqJvEFY0YJKv93Wsg  hflEbENcF1_TC2IoyX_Y4w  q8ZC6BgKh19RkLmH8bJ_Ig   

   stars                                     

In [7]:
import pandas as pd

# Fonction de vérification
def check_issues(df, col='text'):
    return {
        'multi_space': df[col].str.contains(r'\s{2,}').sum(),
        'excessive_punct': df[col].str.contains(r'([!?.,])\1+').sum(),
        'leading_trailing_space': df[col].str.contains(r'^\s+|\s+$').sum(),
        'uppercase_words': df[col].str.contains(r'[A-Z]').sum(),
        'urls': df[col].str.contains(r'http\S+|www\S+').sum(),
        'non_alphanumeric': df[col].str.contains(r'[^a-zA-Z0-9\s!?.,]').sum(),
        'digits': df[col].str.contains(r'\d').sum()
    }

# Fonction de nettoyage
def clean_text(df, col='text'):
    df[col] = (
        df[col]
        .str.strip()  # supprimer espaces début/fin
        .str.lower()  # tout en minuscules
        .str.replace(r'\s{2,}', ' ', regex=True)  # réduire espaces multiples
        .str.replace(r'([!?.,])\1+', r'\1', regex=True)  # réduire ponctuation répétée
        .str.replace(r'http\S+|www\S+', '', regex=True)  # supprimer URL
        .str.replace(r'[\n\t]+', ' ', regex=True)  # remplacer retours à la ligne/tab par espace
        .str.replace(r'[^a-zA-Z0-9\s!?.,]', '', regex=True)  # supprimer caractères spéciaux
        .str.replace(r'\d+', '', regex=True)  # supprimer chiffres
    )
    return df

#  Nettoyage et vérification pour train_reviews 
print("Train reviews avant nettoyage ")
print(check_issues(train_reviews))

train_reviews = clean_text(train_reviews)

print(" Train reviews après nettoyage")
print(check_issues(train_reviews))

# Nettoyage et vérification pour test_reviews 
print("\n Test reviews avant nettoyage ")
print(check_issues(test_reviews))

test_reviews = clean_text(test_reviews)

print("Test reviews après nettoyage")
print(check_issues(test_reviews))

Train reviews avant nettoyage 


/tmp/ipykernel_83/855551558.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  'excessive_punct': df[col].str.contains(r'([!?.,])\1+').sum(),


{'multi_space': 2223066, 'excessive_punct': 1073389, 'leading_trailing_space': 0, 'uppercase_words': 4424750, 'urls': 7158, 'non_alphanumeric': 3424494, 'digits': 1759539}
 Train reviews après nettoyage
{'multi_space': 1825386, 'excessive_punct': 42420, 'leading_trailing_space': 83077, 'uppercase_words': 0, 'urls': 4, 'non_alphanumeric': 0, 'digits': 0}

 Test reviews avant nettoyage 
{'multi_space': 515229, 'excessive_punct': 262800, 'leading_trailing_space': 0, 'uppercase_words': 1148485, 'urls': 1627, 'non_alphanumeric': 871418, 'digits': 446181}
Test reviews après nettoyage
{'multi_space': 457884, 'excessive_punct': 8578, 'leading_trailing_space': 20326, 'uppercase_words': 0, 'urls': 1, 'non_alphanumeric': 0, 'digits': 0}


In [8]:
import pandas as pd

# Fonction de vérification finale
def check_issues(df, col='text'):
    return {
        'multi_space': df[col].str.contains(r'\s{2,}').sum(),
        'excessive_punct': df[col].str.contains(r'([!?.,])\1+').sum(),  # plus d'une répétition
        'leading_trailing_space': df[col].str.contains(r'^\s+|\s+$').sum(),
        'urls': df[col].str.contains(r'http\S+|www\S+').sum()
    }

# Fonction de nettoyage final
def final_clean_text(df, col='text'):
    df[col] = (
        df[col]
        .str.lower()                                 # tout en minuscules
        .str.replace(r'\s+', ' ', regex=True)       # réduire tous les espaces multiples, tabulations et retours à la ligne
        .str.replace(r'([!?.,])\1+', r'\1', regex=True)  # ponctuation répétée → 1 seule occurrence
        .str.replace(r'http\S+|www\S+', '', regex=True)  # supprimer toutes les URLs
        .str.strip()                                 # supprimer espaces début/fin
    )
    return df

#  Nettoyage et vérification pour train_reviews
print("Train reviews avant nettoyage final ")
print(check_issues(train_reviews))

train_reviews = final_clean_text(train_reviews)

print("Train reviews après nettoyage final ")
print(check_issues(train_reviews))

# Nettoyage et vérification pour test_reviews 
print("\n Test reviews avant nettoyage final")
print(check_issues(test_reviews))

test_reviews = final_clean_text(test_reviews)

print(" Test reviews après nettoyage final")
print(check_issues(test_reviews))

Train reviews avant nettoyage final 


/tmp/ipykernel_83/1486112820.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  'excessive_punct': df[col].str.contains(r'([!?.,])\1+').sum(),  # plus d'une répétition


{'multi_space': 1825386, 'excessive_punct': 42420, 'leading_trailing_space': 83077, 'urls': 4}
Train reviews après nettoyage final 
{'multi_space': 0, 'excessive_punct': 0, 'leading_trailing_space': 0, 'urls': 0}

 Test reviews avant nettoyage final
{'multi_space': 457884, 'excessive_punct': 8578, 'leading_trailing_space': 20326, 'urls': 1}
 Test reviews après nettoyage final
{'multi_space': 1, 'excessive_punct': 0, 'leading_trailing_space': 0, 'urls': 0}


In [9]:
for df in [train_reviews, test_reviews]:
    df['user_id'] = df['user_id'].astype('category')
    df['business_id'] = df['business_id'].astype('category')
    df['stars'] = df['stars'].astype('int8')

In [10]:
import gc
gc.collect()

418

In [11]:
all_reviews = pd.concat([train_reviews, test_reviews], ignore_index=True)

In [12]:
import pandas as pd

#  Créer une copie du dataset pour ne pas modifier l'original 
all_reviews_copy = all_reviews.copy()

# Nettoyage rapide 
# Supprimer les textes vides
all_reviews_copy = all_reviews_copy[all_reviews_copy['text'].str.strip() != ""]
# Supprimer les doublons sur user, business et note
all_reviews_copy = all_reviews_copy.drop_duplicates(subset=["user_id", "business_id", "stars"])

# Supprimer les lignes avec valeurs manquantes importantes
all_reviews_copy = all_reviews_copy.dropna(subset=["user_id", "business_id", "stars", "text"])

#  Fonction de filtrage itératif 
def iterative_filter(df, min_user_reviews=5, min_business_reviews=5):
    """
    Filtre le dataset pour garantir :
    - Chaque utilisateur a au moins `min_user_reviews` reviews
    - Chaque business a au moins `min_business_reviews` reviews
    La fonction s'exécute de manière itérative jusqu'à stabilisation.
    """
    while True:
        # Comptage des reviews par user et par business
        user_counts = df['user_id'].value_counts()
        business_counts = df['business_id'].value_counts()
        
        # Filtrage
        filtered_df = df[
            df['user_id'].isin(user_counts[user_counts >= min_user_reviews].index) &
            df['business_id'].isin(business_counts[business_counts >= min_business_reviews].index)
        ]
        
        # Si pas de changement, sortir de la boucle
        if len(filtered_df) == len(df):
            break
        df = filtered_df
    
    return df

#  Appliquer le filtrage 
all_reviews_filtered = iterative_filter(
    all_reviews_copy, 
    min_user_reviews=15, 
    min_business_reviews=15
)

# Vérifications finales 
print("Shape finale :", all_reviews_filtered.shape)
print("Utilisateurs uniques :", all_reviews_filtered['user_id'].nunique())
print("Businesses uniques :", all_reviews_filtered['business_id'].nunique())
print("Reviews totales :", len(all_reviews_filtered))
print("Utilisateur avec le moins de reviews :", all_reviews_filtered['user_id'].value_counts().min())
print("Business avec le moins de reviews :", all_reviews_filtered['business_id'].value_counts().min())

Shape finale : (1148465, 6)
Utilisateurs uniques : 31678
Businesses uniques : 23588
Reviews totales : 1148465
Utilisateur avec le moins de reviews : 15
Business avec le moins de reviews : 15


In [13]:
print("Colonnes disponibles :", all_reviews_filtered.columns.tolist())
num_duplicates = all_reviews_filtered.duplicated(subset=["user_id", "business_id", "stars"]).sum()
print("Nombre de doublons restants :", num_duplicates)
missing_values = all_reviews_filtered[["user_id", "business_id", "stars", "text"]].isna().sum()
print("Valeurs manquantes par colonne :\n", missing_values)
empty_texts = (all_reviews_filtered['text'].str.strip() == "").sum()
print("Nombre de textes vides :", empty_texts)
print("Nombre minimal de reviews par utilisateur :", all_reviews_filtered['user_id'].value_counts().min())
print("Nombre minimal de reviews par business :", all_reviews_filtered['business_id'].value_counts().min())

Colonnes disponibles : ['review_id', 'user_id', 'business_id', 'stars', 'text', 'date']
Nombre de doublons restants : 0
Valeurs manquantes par colonne :
 user_id        0
business_id    0
stars          0
text           0
dtype: int64
Nombre de textes vides : 0
Nombre minimal de reviews par utilisateur : 15
Nombre minimal de reviews par business : 15


In [ ]:
# # Exporter le dataframe en CSV
# all_reviews_filtered.to_csv('all_reviews_filtered.v2.csv', index=False)

In [14]:
import pandas as pd
import zipfile

# Chemin du CSV à sauvegarder
csv_path = "/kaggle/working/all_reviews_filtered.v2.csv"
all_reviews_filtered.to_csv(csv_path, index=False)

# Créer le ZIP
zip_path = "/kaggle/working/all_reviews_filtered.v2.zip"
with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_path, arcname="all_reviews_filtered.v2.csv")  # Nom à l'intérieur du ZIP

print(f"Fichier ZIP créé : {zip_path}")

Fichier ZIP créé : /kaggle/working/all_reviews_filtered.v2.zip


In [ ]:
import re

# Fonction pour analyser les problèmes dans les textes
def analyze_text_issues(df, col='text', long_word_threshold=30):
    return {
        'multi_space': df[col].str.contains(r'\s{2,}').sum(),                          # espaces multiples
        'excessive_punct': df[col].str.contains(r'([!?.,])\1+').sum(),                 # ponctuation répétée
        'long_words': df[col].str.split().apply(
            lambda words: any(len(w) > long_word_threshold for w in words)
        ).sum(),                                                                      # mots trop longs
        'emojis_or_special_chars': df[col].str.contains(r'[^\w\s,\.!?]', regex=True).sum(), # emojis/caractères spéciaux
        'urls': df[col].str.contains(r'http\S+|www\S+').sum(),                         # URLs
        'html_tags': df[col].str.contains(r'<[^>]+>').sum()                             # balises HTML
    }

# Résumé des problèmes
issues_summary = analyze_text_issues(all_reviews_filtered)

print(" Résumé des problèmes dans les textes")
for issue, count in issues_summary.items():
    print(f"{issue}: {count}")

In [ ]:
import re

# Fonction pour détecter les mots répétés successivement
def find_repeated_words(df, col='text'):
    repeated_word_examples = []
    pattern = re.compile(r'\b(\w+)\b(?:\s+\1\b)+', flags=re.IGNORECASE)  # capture répétitions successives
    for idx, text in df[col].items():  # <-- utiliser items() au lieu de iteritems()
        matches = pattern.findall(text)
        if matches:
            repeated_word_examples.append((idx, matches))
    return repeated_word_examples

# Récupérer les exemples
repeated_words_found = find_repeated_words(all_reviews_filtered)

# Afficher quelques exemples
print(f"Nombre de lignes avec mots répétés successivement :", len(repeated_words_found))
print("Exemples :")
for idx, words in repeated_words_found[:5]:  # Affiche 5 premiers exemples
    print(f"Ligne {idx} : {words}")


In [ ]:
import re

# Fonction pour supprimer les répétitions consécutives de mots
def clean_repeated_words(df, col='text'):
    """
    Supprime les répétitions consécutives de mots dans tout le dataset.
    Exemple :
    "good good service" → "good service"
    "I I love this" → "I love this"
    """
    pattern = re.compile(r'\b(\w+)(\s+\1\b)+', flags=re.IGNORECASE)
    df[col] = df[col].str.replace(pattern, r'\1', regex=True)
    return df

# Appliquer sur tout le dataset
all_reviews_filtered = clean_repeated_words(all_reviews_filtered, col='text')

# Vérification globale
print("Nettoyage des mots répétés terminé ")
print("Nombre total de lignes :", len(all_reviews_filtered))
print("Exemple aléatoire :")
print(all_reviews_filtered.sample(1)['text'].values[0])

In [ ]:
import re

# Motif pour détecter les mots répétés consécutifs
pattern = re.compile(r'\b(\w+)(\s+\1\b)+', flags=re.IGNORECASE)

# Vérifier si des répétitions existent encore
repeated_rows = all_reviews_filtered[all_reviews_filtered['text'].str.contains(pattern, na=False)]

if len(repeated_rows) == 0:
    print("Aucune répétition consécutive détectée. Tous les textes sont nettoyés.")
else:
    print(f"Il reste {len(repeated_rows)} lignes avec des répétitions consécutives.")

In [ ]:
import pandas as pd
import zipfile

# Sauvegarder le dataset filtré dans le dossier de travail Kaggle
csv_path = "/kaggle/working/all_reviews_filtered.csv"
all_reviews_filtered.to_csv(csv_path, index=False)

# Créer le ZIP
zip_path = "/kaggle/working/all_reviews_filtered.zip"
with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_path, arcname="all_reviews_filtered.csv")  # arcname définit le nom à l'intérieur du ZIP

print(f"Fichier ZIP créé : {zip_path}")